## Data Colletion from Twitter

In [13]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import numpy as np
import re
import csv
from collections import Counter
import nltk
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [19]:
from datetime import datetime, timedelta

def date_range(start, end):
    start_date = datetime.strptime(start, '%Y-%m-%d').date()
    end_date = datetime.strptime(end, '%Y-%m-%d').date()
    delta = end_date - start_date 
    days = [start_date + timedelta(days=i) for i in range(delta.days + 1)]
    return list(map(lambda n: n.strftime("%Y-%m-%d"), days))

data_collection_period = date_range('2022-01-01', '2022-09-01')

In [15]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('[.*?\-…]',' ', temp)
    temp = re.sub('&amp;','and', temp)
    temp = re.sub("\n"," ", temp)
    temp = re.sub("\t"," ", temp)
    temp = re.sub("[^a-z0-9À-ž ]","", temp)
    temp = temp.split()
    temp = " ".join(word for word in temp)
    return temp
def get_most_recent_tweets(city, amount, distance = '20km'):
    df_city = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance}').get_items(), amount))[['date', 'content']]
    df_city = df_city.rename(columns={"content": "rawContent"})
    # df_city['cleaned_content'] = [clean_tweet(i) for i in df_city['rawContent']]
    df_city.to_csv(f"data/{city}/{amount}_most_recent_tweets.csv")
    return df_city


def get_tweets_over_period(city, amount_per_day, data_collection_period, distance='20km'):
    df_city = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance} since:{data_collection_period[0]} until:{data_collection_period[1]} ').get_items(), amount_per_day))[['date', 'content']]
    for i in tqdm(range(1, len(data_collection_period)-1)):
        df_temp = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance} since:{data_collection_period[i]} until:{data_collection_period[i+1]} ').get_items(), amount_per_day))[['date', 'content']]
        df_city = df_city.append(df_temp)
    df_city = df_city.rename(columns={"content": "rawContent"})
    df_city.to_csv(f"data/{city}/{amount_per_day*len(data_collection_period)}_tweets_over_period.csv")
    return df_city

def clean_tweets_from_file(file_name):
    df_city = pd.read_csv(file_name, index_col=[0])
    df_city['cleaned_content'] = [clean_tweet(i) for i in df_city['rawContent']]
    df_city['id'] = range(1, len(df_city)+1)
    df_city = df_city[['id'] + [x for x in df_city.columns if x != 'id']]
    df_city.set_index('id')
    df_city.to_csv(file_name)
    return df_city


In [22]:
temp_df = clean_tweets_from_file('data/Kingston/24400_tweets_over_period.csv')

In [17]:
# temp_df.head(150)
# clean_tweet("I <3 u")

In [20]:
%%time
# get_most_recent_tweets("Singapore", 3000, distance='20km')
get_tweets_over_period("Accra", 100, data_collection_period)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [19:21<00:00,  4.80s/it]

CPU times: user 20.1 s, sys: 1.36 s, total: 21.5 s
Wall time: 19min 25s


,date,rawContent
0,2022-01-01 23:59:36+00:00,"@AndrewHolnessJM blessed New Year Mr. PM, have Dr. Horace Chang told you that since June when a ZOSO was placed in Norwood St. James no murder was committed there? That goes to show that ZOSOs have a greater impact on curbing crime than SOEs. Central Kingston need a ZOSO NOW"
1,2022-01-01 23:58:44+00:00,@ashamelessgurl Cya pull up fi some?
2,2022-01-01 23:56:48+00:00,@Dark_Emperorr A who fi cook it?
3,2022-01-01 23:53:04+00:00,Very reassuring and genuine. A rallying cry for a better Jamaica for all Jamaicans. https://t.co/Q7Wyd3WOYO
4,2022-01-01 23:52:58+00:00,@roli4eva Indeed .... Luca didn't have a villian either but Encanto was a better watch
...,...,...
95,2022-08-31 22:34:49+00:00,@MayDenn88559924 @jayjaybrown97 Not really the UK £5 note changed to polymer in 2016 - higher denomination notes are being phased in over time. Jamaica is using the same British printers.
96,2022-08-31 22:33:33+00:00,@karlsamudamp Mr Samuda this is what you are saying the Migrant worker ain't living in any bad condition or being treated badly? Please on the next trip send a member of your family and see if they could live out the ordeal. Remember migrant workers leave back their family. You are cruel! https://t.co/4iuyE4HX5r
97,2022-08-31 22:33:17+00:00,@indramistress meeee !!!!
98,2022-08-31 22:30:53+00:00,"@possibilityboss @AdvocateAndria @ChampionPrezzi @rhodenjax @NationwideRadio @Ricaaduh @TaunaThomas I always wondered about that ""allowance """


In [26]:
%%time
get_most_recent_tweets("New York", 3000, distance='20km')

CPU times: user 1.75 s, sys: 85 ms, total: 1.83 s
Wall time: 2min 4s


,date,rawContent,cleaned_content
0,2022-10-04 12:46:15+00:00,@ManotoNews خواهراش میگه که میخوان مث مادرش باشن\n#مهسا_امینی \n#اعتصابات_سراسری \n#OpIran,
1,2022-10-04 12:46:14+00:00,🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡 https://t.co/U8HadmmAO8,
2,2022-10-04 12:46:14+00:00,Simply Be Well -Six Plant Based Body Soaps - Boxed Set NEW https://t.co/AmoTUCY5vY #eBay via @eBay,simply be well six plant based body soaps boxed set new via
3,2022-10-04 12:46:14+00:00,"$IDEX - Ideanomics, Solectrac secured new business contracts to supply electric tractors to major fleet operators https://t.co/3sOiHMyi9S",idex ideanomics solectrac secured new business contracts to supply electric tractors to major fleet operators
4,2022-10-04 12:46:14+00:00,@DrewOnline @reggaeology What petcentage of motorbikes on the road are legally licensed?,what petcentage of motorbikes on the road are legally licensed
...,...,...,...
2995,2022-10-04 12:40:08+00:00,@jeongsteph You’ve been manifesting this haha congrats @jeongsteph !!!,youve been manifesting this haha congrats
2996,2022-10-04 12:40:08+00:00,"@Gokul_Sahni Therein lies the trap of aligning with the West—your prosperity, dependent on Western technology and technology, is subject to adherence to their vision of the world. That vision, or “rules”, codifies Western dominance. Challenge that and you’re finished.",therein lies the trap of aligning with the westyour prosperity dependent on western technology and technology is subject to adherence to their vision of the world that vision or rules codifies western dominance challenge that and youre finished
2997,2022-10-04 12:40:08+00:00,@appare_noka 外で気を付けてね,
2998,2022-10-04 12:40:07+00:00,Putin’s war comes home to Russia | @MID_RF ☠️ #KILLPutinNOW ⚰️ &amp; all his #Oligarchs 🤑 + Kremlin Worms 🪱 &amp; DEAD Men Walking Yevgeny Prigozhin #YevgenyPrigozhin &amp; Ramzan Kadyrov #RamzanKadyrov https://t.co/TKtN39OJMU,putins war comes home to russia amp all his kremlin worms amp dead men walking yevgeny prigozhin amp ramzan kadyrov


In [4]:
clean_tweet("Pallelai bedokwalk no49.misslohahhuat and misskitty u masturbates for victims.this")

'pallelai bedokwalk no49misslohahhuat and misskitty u masturbates for victimsthis'